### Initial Observations

Due to the complex nature of computer security, CybORG's raw observations contain a lot of information presented in a standardised format which takes the form of a series of nested dictionaries and lists. It is recommended you use prettyprint whenever printing a CybORG observation.

We will begin by instantiating CybORG and looking at Red's initial observation.

In [1]:
import inspect
import random
from pprint import pprint
from CybORG import CybORG

path = str(inspect.getfile(CybORG))
path = path[:-10] + '/Shared/Scenarios/Scenario2.yaml'

env = CybORG(path, 'sim')

results = env.reset(agent='Red')
obs = results.observation
pprint(obs)

{'User0': {'Interface': [{'IP Address': IPv4Address('10.0.103.193'),
                          'Interface Name': 'eth0',
                          'Subnet': IPv4Network('10.0.103.192/28')}],
           'Processes': [{'PID': 2020, 'Username': 'SYSTEM'}],
           'Sessions': [{'Agent': 'Red',
                         'ID': 0,
                         'PID': 2020,
                         'Timeout': 0,
                         'Type': <SessionType.RED_ABSTRACT_SESSION: 10>,
                         'Username': 'SYSTEM'}],
           'System info': {'Architecture': <Architecture.x64: 2>,
                           'Hostname': 'User0',
                           'OSDistribution': <OperatingSystemDistribution.WINDOWS_SVR_2008: 4>,
                           'OSType': <OperatingSystemType.WINDOWS: 2>,
                           'OSVersion': <OperatingSystemVersion.W6_1_7601: 13>}},
 'success': <TrinaryEnum.UNKNOWN: 2>}


The dictionary above has two keys: 'success' and 'User0'. The success value indicates whether the previous action ran successfully, or whether it encountered an error. Since this is the start of the scenario, the success value is set to UNKNOWN.

The key 'User0' is a hostid, indicating its corresponding value is data about that host. Here the hostid is equal to the name of the host, altough hostids can also be ip_addresses depending on the previous action.

We will now take a closer look  examine the 'User0' dictionary.

In [2]:
pprint(obs['User0'])

{'Interface': [{'IP Address': IPv4Address('10.0.103.193'),
                'Interface Name': 'eth0',
                'Subnet': IPv4Network('10.0.103.192/28')}],
 'Processes': [{'PID': 2020, 'Username': 'SYSTEM'}],
 'Sessions': [{'Agent': 'Red',
               'ID': 0,
               'PID': 2020,
               'Timeout': 0,
               'Type': <SessionType.RED_ABSTRACT_SESSION: 10>,
               'Username': 'SYSTEM'}],
 'System info': {'Architecture': <Architecture.x64: 2>,
                 'Hostname': 'User0',
                 'OSDistribution': <OperatingSystemDistribution.WINDOWS_SVR_2008: 4>,
                 'OSType': <OperatingSystemType.WINDOWS: 2>,
                 'OSVersion': <OperatingSystemVersion.W6_1_7601: 13>}}


The 'User0' dictionary contains information about the host 'User0'. 'Interfaces' gives us networking information such as the host's ip address, which is randomized each scenario. 'Processes' lets us know any security-relevant processes running on the host that Red knows about; in this case Red begins with a SYSTEM level shell on User0. 'Sessions' lets us know any shells Red is aware of. Again, we can see it only sees its own starting shell. Finally, 'System info' tells us information about the operating system. For example, it is running Windows Server 2008 and has hostname 'User0'.

You may notice that 'Interface', 'Processes' and 'Sessions' all have lists as values. This is because a host can and usually will have multiple of these running at the same time.

We can now look at Blue's initial observation by calling the get_observation() method. Blue has access to the entire network so its initial observation is huge thus we will initially only print out the keys.

In [3]:
blue_obs = env.get_observation('Blue')

print(list(blue_obs.keys()))

['success', 'Defender', 'Enterprise0', 'Enterprise1', 'Enterprise2', 'Op_Host0', 'Op_Host1', 'Op_Host2', 'Op_Server0', 'User0', 'User1', 'User2', 'User3', 'User4']


Printing out Blue's observation for 'User0', we can see it has has a different picture than Red. It has visibility of its own shells, but cannot detect red's.

In [4]:
pprint(blue_obs['User0'])

{'Interface': [{'IP Address': IPv4Address('10.0.103.193'),
                'Interface Name': 'eth0',
                'Subnet': IPv4Network('10.0.103.192/28')}],
 'Processes': [{'PID': 24849, 'Username': 'ubuntu'}],
 'Sessions': [{'Agent': 'Blue',
               'ID': 1,
               'PID': 24849,
               'Timeout': 0,
               'Type': <SessionType.VELOCIRAPTOR_CLIENT: 7>,
               'Username': 'ubuntu'}],
 'System info': {'Architecture': <Architecture.x64: 2>,
                 'Hostname': 'User0',
                 'OSDistribution': <OperatingSystemDistribution.WINDOWS_SVR_2008: 4>,
                 'OSType': <OperatingSystemType.WINDOWS: 2>,
                 'OSVersion': <OperatingSystemVersion.W6_1_7601: 13>},
 'User Info': [{'Groups': [{'GID': 0}], 'Username': 'Administrator'},
               {'Username': 'GreenAgent'},
               {'Groups': [{'GID': 0}],
                'Password': 'vagrant',
                'Username': 'vagrant'},
               {'Username':

### Red and Blue Observations

We will now import a rules-based red agent and run it one step to get a new observation.

In [5]:
from CybORG.Agents import B_lineAgent

action_space = results.action_space
red_obs = results.observation
agent = B_lineAgent()

def step_red(obs):
    action = agent.get_action(obs,action_space)
    print('Red Action:',action)
    print(76*'-')

    results = env.step(action=action,agent='Red')
    obs = results.observation
    pprint(obs)
    
    return results

results = step_red(red_obs)
red_obs = results.observation

Red Action: DiscoverRemoteSystems 10.0.103.192/28
----------------------------------------------------------------------------
{'10.0.103.193': {'Interface': [{'IP Address': IPv4Address('10.0.103.193'),
                                 'Subnet': IPv4Network('10.0.103.192/28')}]},
 '10.0.103.195': {'Interface': [{'IP Address': IPv4Address('10.0.103.195'),
                                 'Subnet': IPv4Network('10.0.103.192/28')}]},
 '10.0.103.199': {'Interface': [{'IP Address': IPv4Address('10.0.103.199'),
                                 'Subnet': IPv4Network('10.0.103.192/28')}]},
 '10.0.103.200': {'Interface': [{'IP Address': IPv4Address('10.0.103.200'),
                                 'Subnet': IPv4Network('10.0.103.192/28')}]},
 '10.0.103.205': {'Interface': [{'IP Address': IPv4Address('10.0.103.205'),
                                 'Subnet': IPv4Network('10.0.103.192/28')}]},
 'success': <TrinaryEnum.TRUE: 1>}


We can see that the agent has performed a 'DiscoverRemoteSystems' action. This represents a network scan discovering active ip addresses on a target subnet. Note these are now the hostids.

Note that the observation only contains information relevant to the previous action. We no longer have information given by the initial observation. This is because observations are designed to represent the output of real tools used by cybersecurity professionals, which only give extremely limited information at any one time. See the Wrapper tutorial for ways of gluing observations together to form a coherent picture.

Meanwhile because we haven't specified a Blue agent, blue team has been passively monitoring the network. However, it doesn't have any tools to detect Red's subnet scan, so its observation is virtually empty.

In [6]:
print(env.get_observation('Blue'))

{'success': <TrinaryEnum.UNKNOWN: 2>}


However, Red's next action is a port scan, which blue is able to detect.

In [7]:
results = step_red(red_obs)
red_obs = results.observation

blue_obs = env.get_observation('Blue')
print(76*'-')
print('Blue Observation:')
print(76*'.')
pprint(blue_obs)
print(76*'.')

Red Action: DiscoverNetworkServices 10.0.103.205
----------------------------------------------------------------------------
{'10.0.103.205': {'Interface': [{'IP Address': IPv4Address('10.0.103.205')}],
                  'Processes': [{'Connections': [{'local_address': IPv4Address('10.0.103.205'),
                                                  'local_port': 22}]},
                                {'Connections': [{'local_address': IPv4Address('10.0.103.205'),
                                                  'local_port': 21}]}]},
 'success': <TrinaryEnum.TRUE: 1>}
----------------------------------------------------------------------------
Blue Observation:
............................................................................
{'User1': {'Interface': [{'IP Address': IPv4Address('10.0.103.205')}],
           'Processes': [{'Connections': [{'local_address': IPv4Address('10.0.103.205'),
                                           'local_port': 22,
                                

Both agents' observations are similar: a flurry of connections to a single ip address as the host is scanned for open ports.

The next action is an exploit establishing a shell for Red on 'User1'. Again both teams have similar observations, observing the connections generated by this.

In [8]:
random.seed(1) # Guarantee Red can be detected when block first executed

results = step_red(red_obs)
red_obs = results.observation


blue_obs = env.get_observation('Blue')
print(76*'-')
print('Blue Observation:')
print(76*'.')
pprint(blue_obs)

Red Action: ExploitRemoteService 10.0.103.205
----------------------------------------------------------------------------
{'10.0.103.193': {'Interface': [{'IP Address': IPv4Address('10.0.103.193')}],
                  'Processes': [{'Connections': [{'local_address': IPv4Address('10.0.103.193'),
                                                  'local_port': 49616,
                                                  'remote_address': IPv4Address('10.0.103.205'),
                                                  'remote_port': 22}]}]},
 '10.0.103.205': {'Interface': [{'IP Address': IPv4Address('10.0.103.205')}],
                  'Processes': [{'Connections': [{'Status': <ProcessState.OPEN: 2>,
                                                  'local_address': IPv4Address('10.0.103.205'),
                                                  'local_port': 22}],
                                 'Process Type': <ProcessType.SSH: 2>},
                                {'Connections': [{'local_addr

Note that there is a 5% chance that Red's exploit fails to register to be detected by Blue. As shown in the example below (at least the first time the block is run).

In [9]:
random.seed(0) # Guarantee Red invisible when block first executed

results = step_red(red_obs)
red_obs = results.observation

blue_obs = env.get_observation('Blue')
print(76*'-')
print('Blue Observation:')
print(76*'.')
pprint(blue_obs)

Red Action: PrivilegeEscalate User1
----------------------------------------------------------------------------
{'Enterprise1': {'Interface': [{'IP Address': IPv4Address('10.0.74.170')}]},
 'User1': {'Interface': [{'IP Address': IPv4Address('10.0.103.205'),
                          'Interface Name': 'eth0',
                          'Subnet': IPv4Network('10.0.103.192/28')}],
           'Sessions': [{'Agent': 'Red',
                         'ID': 1,
                         'Type': <SessionType.SSH: 2>,
                         'Username': 'SYSTEM'}]},
 'success': <TrinaryEnum.TRUE: 1>}
----------------------------------------------------------------------------
Blue Observation:
............................................................................
{'success': <TrinaryEnum.UNKNOWN: 2>}


The final action in this demo sees Red execute a privilege escalation. Red gets a SYSTEM shell, but Blue is unable to see this activity whatsoever.

In [10]:
results = step_red(red_obs)
red_obs = results.observation

blue_obs = env.get_observation('Blue')
print(76*'-')
print('Blue Observation:')
print(76*'.')
pprint(blue_obs)

Red Action: DiscoverNetworkServices 10.0.74.170
----------------------------------------------------------------------------
{'10.0.74.170': {'Interface': [{'IP Address': IPv4Address('10.0.74.170')}],
                 'Processes': [{'Connections': [{'local_address': IPv4Address('10.0.74.170'),
                                                 'local_port': 22}]},
                               {'Connections': [{'local_address': IPv4Address('10.0.74.170'),
                                                 'local_port': 135}]},
                               {'Connections': [{'local_address': IPv4Address('10.0.74.170'),
                                                 'local_port': 3389}]},
                               {'Connections': [{'local_address': IPv4Address('10.0.74.170'),
                                                 'local_port': 445}]},
                               {'Connections': [{'local_address': IPv4Address('10.0.74.170'),
                                             

Fortunately for Blue, Red's Privilege Escalation leaves malware on the system which can be detected using the Analyse action.

In [11]:
from CybORG.Shared.Actions import Analyse

action = Analyse(session=0,agent='Blue',hostname='User1')

results = env.step(action=action,agent='Blue')

pprint(results.observation)

{'User1': {'Files': [{'Density': 0.9,
                      'File Name': 'escalate.exe',
                      'Known File': <FileType.UNKNOWN: 1>,
                      'Known Path': <Path.TEMP: 5>,
                      'Path': 'C:\\temp\\',
                      'Signed': False}]},
 'success': <TrinaryEnum.TRUE: 1>}


Here we see 'User1' has a 'Files' key. The correspending dictionary has been passed through a tool and the 'Density' parameter indicates a high probability this is malware.